In [1]:
import pandas as pd
import numpy as np

In [2]:
np.set_printoptions(suppress=True, precision=2)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
df = pd.read_csv("files/003.csv")

In [4]:
!mkdir -p tmp
df.to_csv("tmp/training_data.csv", index=False)

In [5]:
!cat tmp/training_data.csv

target,a,b,c,d,e,f,g,h
151400,2,2,3,7,37,0,3,0
286000,4,4,6,16,64,0,6,0
172200,2,2,4,24,16,2,3,0
141400,4,4,4,0,32,1,4,0
100800,3,3,4,8,8,0,4,0
9600,0,0,0,0,4,0,0,0
28800,0,0,0,8,0,0,0,0
261800,2,2,2,20,60,5,2,0
118800,0,0,0,8,0,0,0,30
222000,0,0,0,20,0,0,0,50
564400,8,20,20,32,80,5,15,0
434400,8,20,20,38,16,10,10,0
456000,30,10,10,40,40,0,0,0
1028000,40,30,30,100,100,0,0,0
120000,0,0,0,0,0,0,0,40
240000,0,0,0,0,0,0,0,80
216800,20,10,5,0,2,0,0,20
274400,20,10,5,4,20,0,0,20
302800,30,10,5,2,4,0,0,30
418000,30,10,5,10,40,0,0,30
303000,0,0,0,50,50,1,0,0
393000,0,0,0,100,100,1,0,0
634000,10,10,10,80,80,0,10,0
374800,0,15,0,80,0,0,8,0
380000,20,30,40,0,0,0,0,0
604000,50,60,30,0,0,0,0,0
224000,10,20,5,20,0,0,0,0
340000,10,30,5,40,0,0,0,0
28000,0,0,1,0,0,8,0,0
302800,4,4,6,14,74,0,6,0
572000,8,8,12,32,128,0,12,0
344400,4,4,8,48,32,4,6,0
282800,8,8,8,0,64,2,8,0
201600,6,6,8,16,16,0,8,0
19200,0,0,0,0,8,0,0,0
57600,0,0,0,16,0,0,0,0
523600,4,4,4,40,120,10,4,0
237600,0,0,0,16,0,0,0,60
444000,0,0,0

In [6]:
import sagemaker

session = sagemaker.Session()
bucket = session.default_bucket()

In [7]:
s3_path = session.upload_data("tmp/training_data.csv")
s3_path

's3://sagemaker-us-east-1-305262579855/data/training_data.csv'

In [8]:
from sagemaker.inputs import TrainingInput
    
train_input = TrainingInput(s3_path, content_type="text/csv")

In [9]:
!pip install 'sagemaker[local]' --upgrade

In [10]:
!sudo service docker restart

Stopping docker: [  OK  ]
Starting docker:	.[  OK  ]


In [11]:
import os
import sagemaker
from sagemaker import get_execution_role
from sagemaker.local import LocalSession

# How to use Local Mode
sagemaker_session = LocalSession()
sagemaker_session.config = {'local': {'local_code': True}}

role = get_execution_role()

In [12]:
from sagemaker.sklearn.estimator import SKLearn

estimator = SKLearn(entry_point='scripts/sklearn_script.py',
                    session=sagemaker_session,
                    role=role,
                    instance_type='local',
                    instance_count=1,
                    py_version='py3',
                    framework_version='0.20.0')

In [13]:
estimator.fit({'train': train_input})

Creating 4kiuvemr8e-algo-1-ucb0m ... 
Creating 4kiuvemr8e-algo-1-ucb0m ... done
Attaching to 4kiuvemr8e-algo-1-ucb0m
4kiuvemr8e-algo-1-ucb0m | 2021-05-22 11:57:39,564 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
4kiuvemr8e-algo-1-ucb0m | 2021-05-22 11:57:39,567 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
4kiuvemr8e-algo-1-ucb0m | 2021-05-22 11:57:39,579 sagemaker_sklearn_container.training INFO     Invoking user training script.
4kiuvemr8e-algo-1-ucb0m | 2021-05-22 11:57:39,757 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
4kiuvemr8e-algo-1-ucb0m | 2021-05-22 11:57:39,772 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
4kiuvemr8e-algo-1-ucb0m | 2021-05-22 11:57:39,787 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
4kiuvemr8e-algo-1-ucb0m | 2021-05-22 11:57:39,799 sagemaker-training-toolkit INFO     In

In [14]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='local')

Attaching to 0y2jeaaewg-algo-1-xk786
0y2jeaaewg-algo-1-xk786 | Processing /opt/ml/code
0y2jeaaewg-algo-1-xk786 | Building wheels for collected packages: sklearn-script
0y2jeaaewg-algo-1-xk786 |   Building wheel for sklearn-script (setup.py) ... done
0y2jeaaewg-algo-1-xk786 |   Created wheel for sklearn-script: filename=sklearn_script-1.0.0-py2.py3-none-any.whl size=3997 sha256=94316e6acc9cd6d293269b1e71a0bfb0050ae0adcb0d548748f4980455938367
0y2jeaaewg-algo-1-xk786 |   Stored in directory: /tmp/pip-ephem-wheel-cache-yvbd1z9f/wheels/3e/0f/51/2f1df833dd0412c1bc2f5ee56baac195b5be563353d111dca6
0y2jeaaewg-algo-1-xk786 | Successfully built sklearn-script
0y2jeaaewg-algo-1-xk786 | Installing collected packages: sklearn-script
0y2jeaaewg-algo-1-xk786 | Successfully installed sklearn-script-1.0.0
0y2jeaaewg-algo-1-xk786 | /miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importl

In [15]:
# !docker logs npnyj8bzd8-algo-1-fs5z9

In [16]:
result = predictor.predict([[2,2,3,7,37,0,3,0]])
result

array([151438.52])

0y2jeaaewg-algo-1-xk786 | 172.18.0.1 - - [22/May/2021:11:58:01 +0000] "POST /invocations HTTP/1.1" 200 136 "-" "python-urllib3/1.26.4"


### For challenge lab
---

In [17]:
test_dataset = pd.read_csv("tmp/003.test.csv", header = None)

In [18]:
X_test = test_dataset.drop([0], axis = 1)
y_test = test_dataset[0]

In [19]:
X_test.head(10)

,1,2,3,4,5,6,7,8
0,0,0,0,24,0,0,0,0
1,4,4,6,14,74,0,6,0
2,60,20,10,4,8,0,0,60
3,200,240,120,0,0,0,0,0
4,160,200,32,0,0,0,0,0
5,0,0,0,0,24,0,0,0
6,0,0,0,40,0,0,0,100
7,80,40,20,0,8,0,0,80
8,6,12,18,12,144,0,12,0
9,0,0,0,0,0,0,0,80


In [20]:
y_test.head(10)

0      86400
1     302800
2     605600
3    2416000
4    1712000
5      57600
6     444000
7     867200
8     571200
9     240000
Name: 0, dtype: int64

In [21]:
y_pred = predictor.predict(X_test)
y_pred

array([  79771.73,  298613.64,  622576.09, 2515243.03, 1761150.45,
         55819.85,  430836.04,  894681.6 ,  569348.29,  244843.73,
        609331.41,  216746.96,  593632.62,  627732.52,  119650.46,
        662583.23,  449472.5 ,  155280.07,  841745.66,  818582.89,
       1757283.05,  442566.58,  243879.7 ,  931732.45,  992429.28,
       1887498.12,  880869.77,  562584.91, 1616504.95,  251304.86,
        236194.94,  672077.05, 1671070.62, 3638630.95,  397176.21,
        518396.24,  200719.8 ,  810384.17, 1319172.97, 1240888.8 ,
         62710.96,  330294.01,  827584.35,  150035.67,  498346.34,
        235037.54, 1426707.77, 1321928.68,  271254.88,  380986.66,
       1099689.86,  726004.41, 5880812.18,  881196.8 ])

0y2jeaaewg-algo-1-xk786 | 172.18.0.1 - - [22/May/2021:11:58:01 +0000] "POST /invocations HTTP/1.1" 200 560 "-" "python-urllib3/1.26.4"


In [22]:
y_test

0       86400
1      302800
2      605600
3     2416000
4     1712000
5       57600
6      444000
7      867200
8      571200
9      240000
10     627200
11     201600
12     650400
13     630800
14     118800
15     688800
16     433600
17     172800
18     858000
19     858000
20    1824000
21     448000
22     261800
23     908400
24     960000
25    1812000
26     896000
27     572000
28    1520000
29     240000
30     250000
31     650400
32    1838400
33    3804000
34     433600
35     561600
36     216800
37     760000
38    1344000
39    1211200
40      60400
41     340000
42     836000
43     148800
44     480000
45     237600
46    1499200
47    1284000
48     303000
49     380800
50    1097600
51     720000
52    6168000
53     868800
Name: 0, dtype: int64

In [26]:
from sklearn import metrics

mae_8 = metrics.mean_absolute_error(y_test, y_pred)
mse_8 = metrics.mean_squared_error(y_test, y_pred)
rmse_8 =  np.sqrt(metrics.mean_squared_error(y_test, y_pred))
r2_8 = metrics.r2_score(y_test, y_pred)

print(f'Mean Absolute Error: {mae_8}')
print(f'Mean Squared Error: {mse_8}')
print(f'Root Mean Squared Error: {rmse_8}')
print(f'Coefficient of Determination: {r2_8*100}')

Mean Absolute Error: 34247.58220066585
Mean Squared Error: 3641801861.5187864
Root Mean Squared Error: 60347.34345038551
Coefficient of Determination: 99.62826025070277


In [24]:
%store mae_8
%store mse_8
%store rmse_8
%store r2_8

Stored 'mae_8' (float64)
Stored 'mse_8' (float64)
Stored 'rmse_8' (float64)
Stored 'r2_8' (float64)


In [25]:
predictor.delete_endpoint()

Gracefully stopping... (press Ctrl+C again to force)
